## Project Overview

This project aims to enhance a Netflix dataset originally sourced from Kaggle by integrating it with additional rating data from IMDb’s non-commercial datasets. The ultimate goal is to create a more insightful dataset that includes average ratings and number of votes, which will later be used to build an interactive Tableau dashboard for exploratory analysis.

### Data Sources

- **[Netflix Dataset (Kaggle)](https://www.kaggle.com/datasets/shivamb/netflix-shows)**

  The primary dataset contains metadata about movies and TV shows available on Netflix, including fields such as `show_id`, `title`, `type`, `director`, `cast`, `country`, `date_added`, `release_year`, `rating`, `duration`, `listed_in`, and `description`.  
  However, this dataset does **not** include any user ratings or popularity metrics.

- **[IMDb Datasets (Non-Commercial Use Only)](https://developer.imdb.com/non-commercial-datasets/)**  
  To enrich the Netflix dataset with average ratings and popularity indicators, the following IMDb `.tsv.gz` files were utilized:
  - `title.akas.tsv.gz`: Contains alternative titles, localized titles, and region-specific information. Used primarily to help match titles with possible name variations.
  - `title.basics.tsv.gz`: Provides fundamental details for each title (movies, series, etc.), including the primary and original titles, start year, runtime, and genres.
  - `title.ratings.tsv.gz`: Contains the `averageRating` and `numVotes` for each title, serving as the primary source of user engagement metrics.

### Data Processing Steps

1. **Join IMDb datasets**  
   - Merged `title.akas` and `title.basics` on `titleId` / `tconst` to compile comprehensive title information, ensuring localized and original titles are aligned with primary metadata.
   - Further joined the result with `title.ratings` on `tconst` to incorporate `averageRating` and `numVotes`.
     
2. **Align with Netflix dataset**  
   - Performed a join on `title` fields to map IMDb ratings data to the Netflix catalog. This required careful string matching to handle slight differences in title naming conventions across platforms.

3. **Extract final dataset**  
   - Produced a clean dataset containing:
     - `show_id` (from Netflix dataset)
     - `title` (from Netflix dataset)
     - `averageRating` (from IMDb)
     - `numVotes` (from IMDb)
        
4. **Final Table** 
   
|Column|Meaning|
|:-----|:-----|
|show_id| Unique identifier of title|
|title| Movie and TV Show Title|
|averageRating| Average Rating|
|numVotes| Number of Votes|

In [13]:
import pandas as pd

akas = pd.read_csv('title.akas.tsv', sep='\t', dtype=str, na_values='\\N')
basics = pd.read_csv('title.basics.tsv', sep='\t', dtype=str, na_values='\\N')
ratings = pd.read_csv('title.ratings.tsv', sep='\t', dtype=str, na_values='\\N')

#Merge basics + rating on tconst
df = basics.merge(ratings, on='tconst', how='left')

# Merge akas on tconst
df = df.merge(akas, left_on='tconst', right_on='titleId', how='left')

#Keep need columns
result = df[['title', 'averageRating', 'numVotes']]

# drop null averageRating values
result = result.dropna(subset=['averageRating'])

In [5]:
result

,title,averageRating,numVotes
0,Carmencita,5.7,2164
1,Carmencita,5.7,2164
2,Carmencita,5.7,2164
3,Carmencita - spanyol tánc,5.7,2164
4,Καρμενσίτα,5.7,2164
...,...,...,...
52471159,Episodio #3.20,5.8,6
52471160,Folge #3.20,5.8,6
52471161,エピソード #3.20,5.8,6
52471162,Episodio #3.20,5.8,6


In [24]:
#export new file - imdb_titles_with_ratings
result.to_csv('imdb_titles_with_ratings.csv', index=False)

In [28]:
#Load orginal dataset
netflix = pd.read_csv('netflix_title_clean copy.csv', dtype=str)

# remove duplicate show_id of netflix_title_clean (if there is any=)
netflix = netflix.drop_duplicates(subset='show_id', keep='first').reset_index(drop=True)


imdb = pd.read_csv('imdb_titles_with_ratings.csv', dtype=str)

#Check columns name
print("Netflix columns:", netflix.columns)
print("IMDb columns:", imdb.columns)

#join netflix_titles_clean and imdb_titles_with_ratings.csv on title
joined = netflix.merge(imdb, on='title', how='inner')

Netflix columns: Index(['show_id', 'type', 'title', 'date_added', 'release_year', 'rating',
       'content_quality', 'new_movie', 'new_added', 'duration_mins',
       'duration_season'],
      dtype='object')
IMDb columns: Index(['title', 'averageRating', 'numVotes'], dtype='object')


In [32]:
joined

,show_id,type,title,date_added,release_year,rating,content_quality,new_movie,new_added,duration_mins,duration_season,averageRating,numVotes
0,s1,Movie,Dick Johnson Is Dead,2021-09-25,2020,PG-13,Dark Theme - Violence,New,Recently Added,90,NaN,7.4,7489
1,s1,Movie,Dick Johnson Is Dead,2021-09-25,2020,PG-13,Dark Theme - Violence,New,Recently Added,90,NaN,7.4,7489
2,s1,Movie,Dick Johnson Is Dead,2021-09-25,2020,PG-13,Dark Theme - Violence,New,Recently Added,90,NaN,7.4,7489
3,s1,Movie,Dick Johnson Is Dead,2021-09-25,2020,PG-13,Dark Theme - Violence,New,Recently Added,90,NaN,7.4,7489
4,s1,Movie,Dick Johnson Is Dead,2021-09-25,2020,PG-13,Dark Theme - Violence,New,Recently Added,90,NaN,7.4,7489
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143967,s996,Movie,Vizontele,2021-04-23,2001,TV-MA,Light Theme - Non Violence,New,Recently Added,106,NaN,8.0,40310
143968,s996,Movie,Vizontele,2021-04-23,2001,TV-MA,Light Theme - Non Violence,New,Recently Added,106,NaN,8.0,40310
143969,s998,TV Show,Life in Color with David Attenborough,2021-04-22,2021,TV-PG,Light Theme - Non Violence,New,Recently Added,NaN,1,8.2,4299
143970,s998,TV Show,Life in Color with David Attenborough,2021-04-22,2021,TV-PG,Light Theme - Non Violence,New,Recently Added,NaN,1,8.2,4299


In [34]:
#check duplicate show_id
duplicate_counts = netflix['show_id'].value_counts()
print(duplicate_counts[duplicate_counts > 1])

joined = joined.drop_duplicates(subset='show_id', keep='first').reset_index(drop=True)

Series([], Name: count, dtype: int64)


In [42]:
#Keep needed columns
joined = joined[['show_id', 'title', 'averageRating', 'numVotes']]

joined

In [44]:
joined.to_csv('joined_netflix_imdb.csv', index=False)

,show_id,title,averageRating,numVotes
0,s1,Dick Johnson Is Dead,7.4,7489
1,s10,The Starling,6.4,16267
2,s100,On the Verge,5.1,14
3,s1000,Stowaway,6.3,21
4,s1001,Wild Dog,6.1,2508
...,...,...,...,...
7795,s992,One Like It,5.9,97
7796,s994,Shadow and Bone,5.2,53
7797,s995,This Lady Called Life,7.5,77
7798,s996,Vizontele,8.0,40310
